In [1]:
# # Extracting the data
# import tarfile
  
# # open file
# file = tarfile.open('data/wiki_crop.tar')
  
# # print file names
# print(file.getnames())
  
# # extract files
# file.extractall('./')
  
# # close file
# file.close()

In [2]:
import math
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from datetime import datetime
from scipy.io import loadmat

from keras import Input, Model
from keras.applications import InceptionResNetV2
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, Flatten, Dense, BatchNormalization
from keras.layers import Reshape, concatenate, LeakyReLU, Lambda
from keras.layers import Activation, UpSampling2D, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras_preprocessing import image
from keras import backend as K

In [3]:
def calculate_age(taken, dob):
    birth = datetime.fromordinal(max(int(dob) - 366, 1))
    # assume the photo was taken in the middle of the year
    if birth.month < 7:
        return taken - birth.year
    else:
        return taken - birth.year - 1


In [4]:
def load_data(wiki_dir, dataset="wiki"):
     # Load the wiki.mat file
     meta = loadmat(os.path.join(wiki_dir, "{}.mat".format(dataset)))
     # Load the list of all files
     full_path = meta[dataset][0, 0]["full_path"][0]
     # List of Matlab serial date numbers
     dob = meta[dataset][0, 0]["dob"][0]
     # List of years when photo was taken
     photo_taken = meta[dataset][0, 0]["photo_taken"][0] # year
     # Calculate age for all dobs
     age = [calculate_age(photo_taken[i], dob[i]) for i in range(len(dob))]
     # Create a list of tuples containing a pair of an image path and age
     images = []
     age_list = []
     for index, image_path in enumerate(full_path):
          images.append(image_path[0])
          age_list.append(age[index])
     # Return a list of all images and respective age
     return images, age_list
     

## Encoder network
It encodes an image to a latent/hidden vector  (random variable that can't be measured directly)
- latent vectors are attributed to:
    1. We map higher dimensional data to a lower dimensional data with no prior convictions of how the mapping will be done. The NN trains itself for the best configuration.
    2. We cannot manipulate this lower dimensional data. Thus it is "hidden from us.
    3. As we do not know what each dimension means, it is "hidden" from us.
- `Conv2D layer` - It is responsible for extracting features from the input data. It does this by sliding a small kernel over the input data and performing element-wise multiplications with the elements of the filter followed by summing up all of the products. The process is repeated for every position of the filter on the input data resulting in an output called a feature map.
    - Each filter has its own set of weights learned during the training process and are shared across all positions of the input data. e.g. if the input data is an image, a convolutional layer might learn to recognize edges, corners, or other simple patterns in the image. These low-level features can then be combined by higher layers in the model to recognize more complex patterns, such as objects or scenes.
    - hyperparameters:
    1. `filters` = the number of filters to be applied in the layer. Each filter is a small tensor(multi-dimensional array) of weights with the same depth as the input data i.e channels size. Increasing the number of filters increases the capacity of the model to learn complex features from the data, but also increases the number of parameters in the model and the amount of computation required. Choosing the right number of filters is a trade-off between model performance and efficiency.
    2. `Kernel_size` = The size of the kernel being slid across the window. It is a square e.g 5 is (5,5)
    3. `Strides` = How many spaces it shall skip, also a square. An integer or tuple/list of 2 integers, specifying the strides of
    the convolution along the height and width. 
    4. `padding` = addition of extra pixels around the edges of the input data. If the padding parameter is set to 'same', it means that the output feature map has the same spatial dimensions (i.e. height and width) as the input.For example, if the input data is an image with spatial dimensions 64x64, and the kernel size of the convolutional layer is 5x5 with a stride of 2, then padding of 2 rows and 2 columns would be added to the input image so that the output feature map has the same spatial dimensions as the input.
- `LeakyReLu` - activation function that has a small slope for negative input values, rather than zero as in the case of traditional ReLU. LeakyReLU(x) = x for x >= 0 and LeakyReLU(x) = alpha * x for x < 0. Alpha is a small value. Leaky ReLU addresses the dying ReLU problem (where model has an output of 0 for all inputs) by allowing a small gradient for negative input values, which allows the weights of the neuron to be updated and the neuron to "recover." 
- `BatchNormalization` - normalizes the activations of the previous layer at each batch, i.e. it standardizes the mean and variance of the activations. It is typically applied to the outputs of a convolutional or fully connected (FC) layer, before the activation function. After the convolutional layer, the output is passed through a BatchNormalization layer, which normalizes the activations of the previous layer using the mean and variance of the activations, computed across the current batch of training data.
    -  The purpose of batch normalization is to improve the stability and performance of neural networks. It does this by normalizing the activations of the previous layer at each batch, which can help to reduce the internal covariate shift and improve the gradient flow through the network. It has been shown to improve the training and generalization of deep neural networks on a wide range of tasks.
- `Flatten` - It converts this multi-dimensional array into a flat 1D array, with the shape (batch_size, num_features), where num_features is the total number of elements in the original array. The Flatten layer is often used after the convolutional layers of a CNN to prepare the output for the fully connected (FC) layers, which expect a 1D array as input. It is also sometimes used after the pooling layers of a CNN to reduce the spatial resolution of the output and reduce the number of parameters in the model.
    - For example, if the input to the Flatten layer has shape (batch_size, height, width, channels), then the output of the Flatten layer will have shape (batch_size, height * width * channels).
- `Dense` - This is a layer where all the units(neurons) in the previous layer, and the connections are weighted.   
    - When units=1, we are applying a linear transformation unlike when units=4096
    - The purpose of a dense layer is to combine the features learned by the previous layers and make predictions. Dense layers are often used as the output layer of a neural network, but they can also be used in the hidden layers of the network.
    - Increasing the number of units in a dense layer (also called the width of the layer) can increase the capacity of the model, allowing it to learn more complex patterns in the data. However, it can also increase the risk of overfitting, particularly if the model is not regularized properly.
    - On the other hand, decreasing the number of units in a dense layer can decrease the capacity of the model and reduce the risk of overfitting. However, it can also limit the ability of the model to learn complex patterns in the data, which can degrade the performance of the model.
    - In general, it is recommended to start with a small number of units and increase the width of the layer as needed, while keeping an eye on the training and validation error. 
    

In [5]:
# Encoder network
def build_encoder():
    input_layer = Input(64,64,3)

    # 1st Convolutional Block 
    enc = Conv2D(filters=32, kernel_size=5, strides=2, padding='same')(input_layer)
    enc = LeakyReLU(alpha=0.2)(enc)

    # 2nd Convolutional Block
    enc = Conv2D(filters=64, kernel_size=5, strides=2, padding='same')(enc)
    enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # 3rd Convolutional Block
    enc = Conv2D(filters=128, kernel_size=5, strides=2, padding='same')(enc)
    enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # 4th Convolutional Block
    enc = Conv2D(filters=256, kernel_size=5, strides=2, padding='same')(enc)
    enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # Flatten layer
    enc = Flatten()(enc)

    # 1st Fully Connected Layer
    enc = Dense(4096)(enc)
    enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # Second Fully Connected Layer
    enc = Dense(100)(enc)

    # Create a model
    model = Model(inputs=[input_layer], outputs=[enc])

    return model

## Generator network
It is a CNN that takes 100-dimensional vector z and generates an image with a dimension (64,64,3)
- `Droupout` - It is a regularization layer that randomly sets a fraction of the input units to zero during training. 
    - It takes a a fraction of the units to dropout as input, which is specified by the rate parameter
    - E.g. Dropout(0.2) drops 20% of the input units which is equivalent to setting a random 20% of the inputs to zero
    - It is often used after the `Dense` layer to reduce the risk of overfitting
    - During training, the Dropout layer randomly sets a fraction of the input units to zero, which has the effect of reducing the complexity of the model and preventing the units from co-adapting too much.
    - During evaluation or inference, the Dropout layer is usually disabled, and the input is passed through unchanged. This allows the model to make predictions using all the units, which can improve the performance of the model.
- `Upsampling2D` - It increases the resolution of the input tensors by upsampling the data along the spatial (height and width) dimensions. 
    - It is the process of repeating rows a specified number of times x and repeating the columns a specified number of times y
    - `size` = Specifies the upsampling factors along the height and width dimensions. e.g. 2 upsamples the input by a factor of 2 aling both dimensions which is equal to doubling the resolution of the input tensor
    - It is often used in conjunction with convolutional layers to increase the resolution of the feature maps
- `Activation` - It applies element-wise activation function to the input. It helps introduce non-linearity to a NN and allow it to learn more complex patterns in the data
    - `name` = Name of the acivation function to use e.g. tanh, sigmoid etc
    - `tanh activation` = It maps the input to the range [-1,1]. 
    - `sigmoid activation` = It maps the input to the range [0,1]

In [6]:
def build_generator():
    latent_dims = 100
    num_classes = 6
    # Input layer for vector z - 1D
    input_z_noise = Input(shape=(latent_dims, )) # Model expects a tensor with 100 elements as input, TensorShape([None, 100])
    # Input layer for conditioning variable
    input_label = Input(shape=(num_classes, )) #TensorShape([None, 6])
    # Concantenate inputs along the channel dimension and generate a concatenated tensor
    x = concatenate([input_z_noise, input_label]) # TensorShape([None, 106])

    # 1st Desnse layer
    x = Dense(2048, input_dim=latent_dims+num_classes)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)

    # 2nd Dense layer
    x = Dense(256 * 8 * 8)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)

    # Reshape the output to a 3D tensor with dimensions (8,8,256)
    x = Reshape((8, 8, 256))(x) # output is a tensor (batch_size, 8,8,256)

    # 1st Upsampling layer
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(filters=128, kernel_size=5, padding='same')(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = LeakyReLU(alpha=0.2)(x)

    # 2nd Upsampling layer
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(filters=64, kernel_size=5, padding='same')(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = LeakyReLU(alpha=0.2)(x)

    # 3rd Upsampling layer
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(filters=3, kernel_size=5, padding='same')(x)
    x = Activation('tanh')(x)

    model = Model(inputs=[input_z_noise, input_label], outputs=[x])
    return model


## Discriminator Network
It is a CNN that tries to differentiate between the real data and the data generated by the generator network. The discriminator network tries to put the incoming data into predefined categories. It can either perform multi-class classification or binary classification. Generally, in GANs binary classification is performed.It processes an image and outputs a probability of the image belonging to a particular class



In [7]:
# The expand_label_input function
def expand_label_input(x):
    """
    It takes an input tensor x and expands its dimensions 
    to match the shape of the target tensor
    The expand_dims function is used to insert a new axis at 
    a specified position in the tensor
    The axis parameter specifies the position of the new axis 
    with 0 == first axis and K.ndim(x)-1 to the last axis
    The function is expanding the input tensor x aling the first
    and second axes by inserting two new axes at position 1 and 2
    The tile function is used to repeat the input tensor along 
    specified dimensions. 
    The function repeats the input tensor along the first,second
    and fourth dimensions to match the shape of the target tensor
    with shape (batch_size, 32,32,num_channels)
    """
    x = K.expand_dims(x, axis=1)
    x = K.expand_dims(x, axis=1)
    x = K.tile(x, [1, 32, 32, 1])
    return x

In [8]:
def build_discriminator():
    # Input image shape
    input_shape = (64, 64, 3)
    # Input conditioning variable shape
    label_shape = (6,)
    # Two input layers
    image_input = Input(shape=input_shape) # TensorShape([None, 64, 64, 3])
    label_input = Input(shape=label_shape) #TensorShape([None, 6])

    # 1st Convolution layer
    x = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(image_input)
    x = LeakyReLU(alpha=0.2)(x)

    # Expand label_input to have a shape of (32,32,6)
    label_input1 = Lambda(expand_label_input)(label_input) # TensorShape([None, 32, 32, 6])

    # Concatenate the transformed label tensor and the output of the last convolution layer
    x = concatenate([x, label_input1], axis=3) # TensorShape([None, 32, 32, 70])

    # 2nd Convolution layer
    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    # 3rd Convolution layer
    x = Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    # 4th Convolution layer
    x = Conv2D(512, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    # Flatten layer
    x = Flatten()(x)

    # Dense layer
    x = Dense(1, activation='sigmoid')(x) # outputs a probability

    model = Model(inputs=[image_input, label_input], outputs=[x])
    return model

## Training the cGAN
Three steps involved:
    - Training the cGAN - the NNs
    - Initial latent vector approximation
    - Latent vector optimization
    

### Step 1: Training the cGAN
- `Adam` - It is an optimization algorithm that is used to update the parameters (weights) of a neural network during training
    - It uses an adaptive learning rate that decreases over time. It estimates the mean and variance of the gradient for eaach parameter and upates the parameters. 
    -  The gradient is a vector that points in the direction of the steepest increase of the loss function and has the same shape as the parameters. The optimizer uses the gradient to adjust the parameters in the opposite direction, which should reduce the value of the loss function.
    - lr=0.0002: The `learning rate`.
    - beta_1=0.5: The `exponential decay rate` for the mean.
    - beta_2=0.999: The `exponential decay rate` for the variance.
    - epsilon=10e-8: The small value used to `prevent division by zero.`
- `Adversarial model` - is a combination of the generator and discriminator
    - The adversarial model is trained by alternating between training the generator to generate better fake images and training the discriminator to better distinguish between real and fake images.
    - The `discriminator.trainable` attribute is set to `False`, which means that the weights of the discriminator model will not be updated during training of the adversarial model. This is because the goal of the adversarial model is to train the generator to generate better fake images, not to update the weights of the discriminator.
    - The adversarial model takes in the `input_z_noise` and `input_label` tensors as input and `outputs the valid tensor`.
- `Normalization` - Ithelps the GAN model to learn more effectively, as the model will be able to learn from data that is centered around 0 and has small values, rather than large values that may be harder for the model to learn from.
    - Normalization can help the model converge faster and perform better. 
    - In this specific case, the images are being normalized by scaling them to a range of [-1, 1]. This is often done in machine learning when the input data has values in a different range, such as [0, 255] for pixel values of images, in order to make the data easier to work with and improve model performance. It is not necessary to always subtract by 1, it just depends on the range of the input data and what is most appropriate for the specific task.

In [9]:
def age_to_category(ages):
    """ 
    Function to convert the ages into different categories
    """
    ages_list = []
    for age in ages:
        if 0 < age <= 18:
            age_category = 0
        elif 18 < age <= 29:
            age_category = 1
        elif 29 < age <= 39:
            age_category = 2
        elif 39 < age <= 49:
            age_category = 3
        elif 49 < age <= 59:
            age_category = 4
        elif age >= 60:
            age_category = 5
        ages_list.append(age_category)
    return ages_list


In [10]:
def load_images(data_dir, image_paths, image_shape):
    """ 
    This function loads a set of images from a given directory,
    resizes them to the given shape and concantenates them into 
    a single tensor
    """
    images = None
    for i, image_path in enumerate(image_paths):
        try:
            # Load image
            loaded_image = image.load_img(os.path.join(data_dir, image_path), target_size=image_shape)
            # Convert PIL image to numpy ndarray
            loaded_image = image.img_to_array(loaded_image)
            # Add another dimension (Add batch dimension)
            loaded_image = np.expand_dims(loaded_image, axis=0)
            # Concatenate all images into one tensor
            if images is None:
                images = loaded_image
            else:
                # print(False)
                images = np.concatenate([images, loaded_image], axis=0)
        except Exception as e:
            print("Error:", i, e)
    return images

In [11]:
def save_rgb_img(img, path):
    """
    Save a rgb image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")
    plt.savefig(path)
    plt.close()

In [12]:
def write_log(callback, name, value, batch_no):
  writer = tf.summary.create_file_writer("/tmp/mylogs")
  with writer.as_default():
    for step in range(100):
      # other model code would go here
      tf.summary.scalar("my_metric", 0.5, step=step)
      writer.flush()

In [13]:
# data_dir = r"C:/Users/meshw/Desktop/Moringa/phase_5/project/Wiki_crop"
# wiki_dir = os.path.join(data_dir, "wiki_crop")

# Define hyperparameters
data_dir = os.getcwd()
wiki_dir = os.path.join(data_dir, "wiki_crop")
epochs = 500
batch_size = 128
image_shape = (64, 64, 3)
z_shape = 100
TRAIN_GAN = True
TRAIN_ENCODER = False
TRAIN_GAN_WITH_FR = False
fr_image_shape = (192, 192, 3)


In [14]:
epochs = 20
batch_size = 2

In [ ]:
# Define optimizers
from tensorboard import summary
# Optimizer for the discriminator network
dis_optimizer = Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999, epsilon=10e-8)
# Optimizer for the generator network
gen_optimizer = Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999, epsilon=10e-8)
# Optimizer for the adversarial network
adversarial_optimizer = Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999, epsilon=10e-8)

# Build and compile the discriminator network
discriminator = build_discriminator()
discriminator.compile(loss=['binary_crossentropy'], optimizer=dis_optimizer)
# Build and compile the generator network
generator = build_generator()
generator.compile(loss=['binary_crossentropy'], optimizer=gen_optimizer)

# Build and compile the adversarial model
discriminator.trainable = False # it is important to ensure the discriminator is not training when the generator is training
input_z_noise = Input(shape=(100,)) # TensorShape([None, 100])
input_label = Input(shape=(6,)) # TensorShape([None, 6])
recons_images = generator([input_z_noise, input_label]) # These are the fake images generated by the generator network of shape TensorShape([None, 64, 64, 3])
valid = discriminator([recons_images, input_label]) 
adversarial_model = Model(inputs=[input_z_noise, input_label], outputs=[valid])
adversarial_model.compile(loss=['binary_crossentropy'], optimizer=gen_optimizer)

# Using Tensorboard to store losses
tensorboard = TensorBoard(log_dir="logs/{}".format(time.time()))
tensorboard.set_model(generator)
tensorboard.set_model(discriminator)

# Load images and ages from the dataset
images, age_list = load_data(wiki_dir=wiki_dir, dataset="wiki")
    
# Convert age to category
age_cat = age_to_category(age_list)

# Convert the age categories to one-hot encoded vectors
final_age_cat = np.reshape(np.array(age_cat), [len(age_cat), 1]) # Convert the shape to (62328, 1)
classes = len(set(age_cat)) # Store unique classes
y = to_categorical(final_age_cat, num_classes=len(set(age_cat))) # Returns a 2D array containing the one-hot encoded vectors and shape (62328, 7) 

# Read all images and create an ndarray
loaded_images = load_images(wiki_dir, images[:20], (image_shape[0], image_shape[1]))

## Implement label smoothing
real_labels = np.ones((batch_size, 1), dtype = np.float32) * 0.9
fake_labels = np.zeros((batch_size, 1), dtype = np.float32) * 0.1
# Epochs
for epoch in range(epochs):
    print("Epoch:{}".format(epoch))
    gen_losses = []
    dis_losses = []
    number_of_batches = int(len(loaded_images) / batch_size)
    print("Number of batches:", number_of_batches)
    for index in range(number_of_batches):
        print("Batch:{}".format(index + 1))
        images_batch = loaded_images[index * batch_size:(index + 1) * batch_size]
        images_batch = (images_batch / 255.0 - 0.5) / 0.5 # Normalizes the pixel values to be between 1 and -1
        images_batch = images_batch.astype(np.float32) # Converts data type to float
        y_batch = y[index * batch_size:(index + 1) * batch_size]
        z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))
        initial_recon_images = generator.predict_on_batch([z_noise, y_batch])
        d_loss_real = discriminator.train_on_batch([images_batch, y_batch], real_labels)
        d_loss_fake = discriminator.train_on_batch([initial_recon_images, y_batch], fake_labels)
        # Again sample a batch of noise vectors from a Gaussian(normal) distribution 
        z_noise2 = np.random.normal(0, 1, size=(batch_size, z_shape))
        # Samples a batch of random age values
        random_labels = np.random.randint(0, 6, batch_size).reshape(-1, 1)
        # Convert the random age values to one-hot encoders
        random_labels = to_categorical(random_labels, 6)
        
        # Train the generator network
        g_loss = adversarial_model.train_on_batch([z_noise2, random_labels], np.ones((batch_size,1))) # Sets the likelihood that ist predictions are equal to 1

        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        print("d_loss:{}".format(d_loss))
        print("g_loss:{}".format(g_loss))


        write_log(tensorboard, 'g_loss', np.mean(gen_losses), epoch)
        write_log(tensorboard, 'd_loss', np.mean(dis_losses), epoch)

        if epoch % 10 == 0:
            images_batch = loaded_images[0:batch_size]
            images_batch = (images_batch / 255.0 - 0.5) / 0.5
            images_batch = images_batch.astype(np.float32)
            y_batch = y[0:batch_size]
            z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))
            gen_images = generator.predict_on_batch([z_noise, y_batch])
            for i, img in enumerate(gen_images[:5]):
                save_rgb_img(img, path=os.path.join(os.getcwd(), "results/img_{}_{}.png".format(epoch, i)))

        # Save weights only
        generator.save_weights("generator.h5")
        discriminator.save_weights("discriminator.h5")
        # Save architecture and weights both
        generator.save("generator.h5")
        discriminator.save("discriminator.h5")


In [ ]:
# # Epochs
# for epoch in range(epochs):
#     print("Epoch:{}".format(epoch))
#     gen_losses = []
#     dis_losses = []
#     number_of_batches = int(len(loaded_images) / batch_size)
#     print("Number of batches:", number_of_batches)
#     for index in range(number_of_batches):
#         print("Batch:{}".format(index + 1))
#         images_batch = loaded_images[index * batch_size:(index + 1) * batch_size]
#         images_batch = (images_batch / 255.0 - 0.5) / 0.5 # Normalizes the pixel values to be between 1 and -1
#         images_batch = images_batch.astype(np.float32) # Converts data type to float
#         y_batch = y[index * batch_size:(index + 1) * batch_size]
#         z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))
#         initial_recon_images = generator.predict_on_batch([z_noise, y_batch])
#         d_loss_real = discriminator.train_on_batch([images_batch, y_batch], real_labels)
#         d_loss_fake = discriminator.train_on_batch([initial_recon_images, y_batch], fake_labels)
#         # Again sample a batch of noise vectors from a Gaussian(normal) distribution 
#         z_noise2 = np.random.normal(0, 1, size=(batch_size, z_shape))
#         # Samples a batch of random age values
#         random_labels = np.random.randint(0, 6, batch_size).reshape(-1, 1)
#         # Convert the random age values to one-hot encoders
#         random_labels = to_categorical(random_labels, 6)
        
#         # Train the generator network
#         g_loss = adversarial_model.train_on_batch([z_noise2, random_labels], np.ones((batch_size,1))) # Sets the likelihood that ist predictions are equal to 1
        
#         # Calculate and print the losses
#         d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
#         print("d_loss:{}".format(d_loss))
#         print("g_loss:{}".format(g_loss))
        
#         # Add losses to their respective lists
#         gen_losses.append(g_loss)
#         dis_losses.append(d_loss)

#         # Write losses to Tensorboard for visualization
#         write_log(tensorboard, 'g_loss', np.mean(gen_losses), epoch)
#         write_log(tensorboard, 'd_loss', np.mean(dis_losses), epoch)

#         # Sample and save images after every 10 epochs
#         if epoch % 10 == 0:
#             images_batch = loaded_images[0:batch_size]
#             images_batch = images_batch / 127.5 - 1.0
#             images_batch = images_batch.astype(np.float32)
#             y_batch = y[0:batch_size]
#             z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))
#             gen_images = generator.predict_on_batch([z_noise, y_batch])
#             for i, img in enumerate(gen_images[:5]):
#                 save_rgb_img(img, path="results/img_{}_{}.png".format(epoch, i))

#         # Save weights only
#         generator.save_weights("generator.h5")
#         discriminator.save_weights("discriminator.h5")
#         # Save architecture and weights both
#         generator.save("generator.h5")
#         discriminator.save("discriminator.h5")

## Initial latent vector approximation